In [ ]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pymongo import MongoClient
import sys
import base64

from AnimalShelter import AnimalShelter



###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "1234"
shelter = AnimalShelter()
sys.setrecursionlimit(2000)


# class read method must support return of cursor object and accept projection json input
df = pd.DataFrame.from_records(shelter.read({}))
dff = px.data.tips()
del df["_id"]



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Jessica Amos, SNHU 2022')

logo_png = 'logo.png'
display_image = base64.b64encode(open(logo_png, 'rb').read())

app.layout = html.Div([
    #     #Grazioso Salvare’s logo
    html.Img(src='data:logo.png;base64,{}'.format(display_image.decode(), 
            style = {
                'width':'5%', 
                'height':'5%', 
                'float': 'right'})),
    
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Austin Animal Shelter Center Outcomes'))),
    html.Center(html.P('Jessica Amos, SNHU 2022')),
#     html.Img(id='customer-image',src='data:image/png;base64,{}'.format(display_image.decode()),alt='customer image'),
    # Button location at top
    html.Div(className='row',
             style={'display' : 'flex'},
                  children=[
                          html.Button(id='submit-button-one', n_clicks=0, children='Mountain Wilderness'),
                          html.Button(id='submit-button-two', n_clicks=0, children='Disaster / Indiv. Tracking'),
                          html.Button(id='submit-button-two', n_clicks=0, children='Water Rescue')
                  ]),

#FIXME Add in code for the interactive filtering options. For example, Radio buttons, drop down, checkboxes, etc.

    html.Div([
        dash_table.DataTable(
            id='datatable-id',
            columns=[
                {"name": i,"id": i, "deletable": False, "selectable": True} for i in df
            ],


            data=df.to_dict('records'),
            # features for interactive data table to make it user-friendly for your client
            editable =False,
            sort_action='native',
            sort_mode = 'multi',
            filter_action='native',
            column_selectable = True,
            row_selectable = 'multi',
            row_deletable=False,
            selected_columns=[],
            selected_rows=[],
            page_action='native',
            page_current=0,
            page_size=5,
            
            style_cell=
            {
                'textAlign':'left',
                'padding':'12px'
            },
            style_as_list_view = True,

            style_header=
            {
                'backgroundColor':'rgb(210,210,210)',
                'textAlign':'justify',
                'padding':'15px'
            },


            ),
      ]), 
    
    #displays geoMap
    html.Br(),
    html.Hr(),
    html.Div(
        id = 'map-id',
        className = 'col s12 m6',
    ),
    html.Div([
#     html.P("Rescue Type:"),
#     dcc.Dropdown(
#         id='animal_type', 
#         value='animal_type', 
#         options=[{'value': x, 'label': x} 
#                  for x in ['']],
#         clearable=False
#     ),
#     html.P("Values:"),
#     dcc.Dropdown(
#         id='values', 
#         value='total_bill', 
#         options=[{'value': x, 'label': x} 
#                  for x in ['total_bill', 'tip', 'size']],
#         clearable=False
#     ),
    dcc.Graph(id="pie-chart"),
])

])
#############################################
# Interaction Between Components / Controller
#############################################




# def update_data
@app.callback(Output('datatable-id',"data"), 
              [Input('submit-button-one', 'n_clicks'),Input('submit-button-two', 'n_clicks')
              ])
        
def on_click(bt1,bt2):
    # start case
    if (int(bt1) == 0 and int(bt2) == 0):
        df = pd.DataFrame.from_records(shelter.readAll({}))
    # use higher number of button clicks to determine filter type, can you think of a better way ? ....
    elif (int(bt1) > int(bt2)):
        df = pd.DataFrame(list(shelter.readAll({"animal_type":"Cat"})))
    elif (int(bt2) > int(bt1)):
        df = pd.DataFrame(list(shelter.readAll({"animal_type":"Dog"})))
    
        
    return df.to_dict('records')


# changes background color of selected items
def update_styles(selected_columns):
    return [{'if': { 'column_id': i }, 'background_color': '#D2F3FF'} for i in selected_columns]

# # calls the data from the table into the map
@app.callback(
    Output('map-id',"children"),
    [Input('datatable-id',"derived_viewport_data")])

# geo-location map
def update_map(viewData):
    if viewData is None:
        return []
    else:
        dff = pd.DataFrame.from_dict(viewData)
        # Austin TX is at [30.75,-97.48]
        return [
            dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
                dl.TileLayer(id="base-layer-id"),
                # Marker with tool tip and popup
                dl.Marker(position=[30.75,-97.48], children=[
                    dl.Tooltip(dff.iloc[0,4]),
                    dl.Popup([
                        html.H1("Animal Name"),
                        html.P(dff.iloc[1,9])
                    ])
                ])
            ])
        ]
@app.callback(
    Output("pie-chart", "figure"), 
    [Input('datatable-id',"derived_viewport_data")])
def generate_chart(viewData):
    fig = px.pie(df, values=values, names=name)
    return fig
    
app